In [2]:
import sys, os, pathlib
#setting the path to folder with modules
sys.path.insert(0, str(pathlib.Path(os.getcwd()).parents[1] / 'python'))
from Load_Datasets import *

In [50]:
#basic imports
import sys, os, copy
import numpy as np
import datetime as dt
import pandas as pd

#import libraries for file checking
from os.path import isfile, join, isdir

#importing json (data are mostly in JSON)
import json

#For text mining
from stemming.porter2 import stem
from tqdm import tqdm
import math
import collections
import scipy

In [130]:
FileAddress_movies ="../../Datasets/tmdb_5000_movies.csv"
FileAddress_credits="../../Datasets/tmdb_5000_credits.csv"

In [ ]:
#Loading information about movies
Movies,Credentials  = Load_Datasets(FileAddress_movies,FileAddress_credits)

Drops = ['homepage','status','id']
for drop in Drops:
    Movies = Movies.drop(drop, 1)
        
    

Movies.to_csv("../../Datasets/Transformed.csv")
#print(Movies.dtypes)
#read arrays from string using string.split(",")

In [5]:
Movies = pd.read_csv("../../Datasets/Transformed.csv")
Movies

,title,budget,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,tagline,vote_average,vote_count,actors,actor_gender,crew_names,crew_jobs,crew_departments
0,#Horror,1500000,"Drama,Mystery,Horror,Thriller",NaN,de,#Horror,"Inspired by actual events, a group of 12 year ...",2.815228,"AST Studios,Lowland Pictures",United States of America,...,90.0,"English,Español",Death is trending.,3.3,52,"Taryn Manning,Natasha Lyonne,Chloë Sevigny,Bal...","1,1,1,2,2,1,0,0,1","Tara Subkoff,Tara Subkoff,Tara Subkoff,Jason L...","Screenplay,Director,Producer,Producer,Producer...","Writing,Directing,Production,Production,Produc..."
1,(500) Days of Summer,7500000,"Comedy,Drama,Romance","date,sex,jealousy,fight,architect,gallery,inte...",en,(500) Days of Summer,"Tom (Joseph Gordon-Levitt), greeting-card writ...",45.610993,"Fox Searchlight Pictures,Watermark,Dune Entert...",United States of America,...,95.0,"English,Français,svenska",It was almost like falling in love.,7.2,2904,"Joseph Gordon-Levitt,Zooey Deschanel,Chloë Gra...","2,1,1,2,2,2,1,1,1,2,2,1,1,0,2,1,0,0,1,0,0,2,0","Mychael Danna,Hope Hanafin,Steven J. Wolfe,Mas...","Original Music Composer,Costume Design,Produce...","Sound,Costume & Make-Up,Production,Production,..."
2,10 Cloverfield Lane,15000000,"Thriller,Science Fiction,Drama","kidnapping,bunker,paranoia,basement,survivalis...",en,10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",53.698683,"Paramount Pictures,Bad Robot,Spectrum Effects",United States of America,...,103.0,English,Monsters come in many forms.,6.8,2468,"Mary Elizabeth Winstead,John Goodman,John Gall...","1,2,2,2,2,0,1,0,2,0","Monika Mikkelsen,J.J. Abrams,Matthew W. Mungle...","Casting,Producer,Makeup Effects,Director of Ph...","Production,Production,Crew,Camera,Sound,Art,Cr..."
3,10 Days in a Madhouse,1200000,Drama,"undercover,insane asylum,reporter",en,10 Days in a Madhouse,"Nellie Bly, a 23 year-old reporter for Joseph ...",0.489271,NaN,United States of America,...,111.0,English,NaN,4.3,5,"Caroline Barry,Christopher Lambert,Kelly LeBro...","0,2,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,2,0","Martin Wiley,Jan Glaser,Strathford Hamilton,Ma...","Producer,Casting,Executive Producer,Executive ...","Production,Production,Production,Production,Di..."
4,10 Things I Hate About You,16000000,"Comedy,Romance,Drama","shakespeare,sister,high school,cannabis,decept...",en,10 Things I Hate About You,"Bianca, a tenth grader, has never gone on a da...",54.550275,"Mad Chance,Jaret Entertainment,Touchstone Pict...",United States of America,...,97.0,"English,Français",How do I loathe thee? Let me count the ways.,7.3,1701,"Heath Ledger,Julia Stiles,Joseph Gordon-Levitt...","2,1,2,1,2,2,1,1,2,2,1,2,2,0,2,0,2,0,0,0,0,1,0,...","Charles Graffeo,William Shakespeare,Mark Irwin...","Set Decoration,Theatre Play,Director of Photog...","Art,Writing,Camera,Editing,Costume & Make-Up,S..."
5,102 Dalmatians,85000000,"Comedy,Family","london england,prison,release from prison,wome...",en,102 Dalmatians,Get ready for a howling good time as an all ne...,9.895061,"Walt Disney Pictures,Cruella Productions",United States of America,...,100.0,English,Meet Two Unlikely Heroes With A Bone To Pick.,5.1,313,"Glenn Close,Ioan Gruffudd,Alice Evans,Tim McIn...","1,2,1,2,2","David Newman,Adrian Biddle,Kevin Lima,Gregory ...","Music,Director of Photography,Director,Editor,...","Sound,Camera,Directing,Editing,Production,Writ..."
6,10th & Wolf,8000000,"Action,Crime,Drama,Mystery,Thriller","undercover,mafia,mobster,crime family",en,10th & Wolf,A former street tough returns to his Philadelp...,3.942464,Thinkfilm,United States of America,...,107.0,English,"The Intersection Where Family, Honor and Betra...",6.3,24,"James Marsden,Brian Dennehy,Leo Rossi,Dennis H...","2,2,2,2,2,2,2,2,2,2,2,2,1","Robert Moresco,Robert Moresco,Allan Steele","Director,Writer,Writer","Directing,Writing,Writing"
7,11:14,6000000,"Crime,Drama,Thriller

In [81]:
def cost_function():
    """Define the cost between the film given in index and the others one."""
    costs = np.zeros([Movies.shape[0],Movies.shape[0]])
    
    for i in tqdm(range(Movies.shape[0])):
        current_film = Movies.iloc[i]
        genres_current = get_genres(current_film)
        kw_current = get_keywords(current_film)

        for j in range(i,Movies.shape[0]):
            cost = 0

            b_film = Movies.iloc[j]
            genres_b = get_genres(b_film)
            #First we only select the first genre to determine the similarity because it's more important that the other genre.
            if len(genres_current) > 0  & len(genres_b) > 0:
                if (genres_current[0] == genres_b[0]):
                    cost += 2

                #This give us the number of similar genres. We pop the first one because we already compare them.
                cost += np.sum(np.in1d(genres_current,genres_b.pop(0),assume_unique='True')) / 10
                #TODO: Do the same for secondary genre but with less weight.


            kw_b = get_keywords(b_film)
            #This give us the number of similar keywords.
            cost += np.sum(np.in1d(kw_current,kw_b,assume_unique='True'))

            costs[i,j] = cost
            costs[j,i] = cost
    return costs

def get_genres(film):
    genres = str(film['genres'])
    if genres == 'nan':
        return[]
    else:
        genres = genres.split(",")
    return genres

def get_keywords(film):
    kw = str(film['keywords'])
    if kw == 'nan':
        return[]
    else:
        kw = kw.split(",")
    return kw

def text_mining():
    cosine_distances = np.zeros([Movies.shape[0],Movies.shape[0]])
    
    for m in tqdm(range(Movies.shape[0])):
        current_words = get_film_words(Movies.iloc[m])

        for n in range(m, Movies.shape[0]):
            
            words = get_film_words(Movies.iloc[n])
            cosine = get_cosine(current_words,words)
            
            cosine_distances[m,n] = cosine
            cosine_distances[n,m] = cosine
            
    return cosine_distances
            
def get_film_words(film):
    stop_words = ['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its', 'before', 'herself', 'had', 'should', 'to', 'only', 'under', 'ours', 'has', 'do', 'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'did', 'this', 'she', 'each', 'further', 'where', 'few', 'because', 'doing', 'some', 'are', 'our', 'ourselves', 'out', 'what', 'for', 'while', 'does', 'above', 'between', 't', 'be', 'we', 'who', 'were', 'here', 'hers', 'by', 'on', 'about', 'of', 'against', 's', 'or', 'own', 'into', 'yourself', 'down', 'your', 'from', 'her', 'their', 'there', 'been', 'whom', 'too', 'themselves', 'was', 'until', 'more', 'himself', 'that', 'but', 'don', 'with', 'than', 'those', 'he', 'me', 'myself', 'these', 'up', 'will', 'below', 'can', 'theirs', 'my', 'and', 'then', 'is', 'am', 'it', 'an', 'as', 'itself', 'at', 'have', 'in', 'any', 'if', 'again', 'no', 'when', 'same', 'how', 'other', 'which', 'you', 'after', 'most', 'such', 'why', 'a', 'off', 'i', 'yours', 'so', 'the', 'having', 'once']
    
    overview = str(film['overview'])
    if overview == 'nan':
        return collections.Counter([])
    
    words = [i for i in film['overview'].lower().split() if i not in stop_words]
        
    for word in words:
        word = stem(word)

    return collections.Counter(words)

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator


In [32]:
cosine_distances = text_mining()

print(cosine_distances)

100%|██████████| 4809/4809 [5:29:39<00:00,  4.11s/it]  


[[ 1.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          1.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          1.         ...,  0.02505487  0.          0.02338262]
 ..., 
 [ 0.          0.          0.02505487 ...,  1.          0.06506     0.03456506]
 [ 0.          0.          0.         ...,  0.06506     1.          0.03035884]
 [ 0.          0.          0.02338262 ...,  0.03456506  0.03035884  1.        ]]


In [33]:
np.savetxt('cosine.csv',cosine_distances,delimiter=',')

In [83]:
costs = cost_function()
        
print(costs)

100%|██████████| 4809/4809 [53:45<00:00,  1.49it/s]  

[[  0.   0.   0. ...,   0.   0.   0.]
 [  0.  17.   0. ...,   0.   0.   0.]
 [  0.   0.   8. ...,   0.   0.   0.]
 ..., 
 [  0.   0.   0. ...,  10.   0.   0.]
 [  0.   0.   0. ...,   0.  11.   0.]
 [  0.   0.   0. ...,   0.   0.  16.]]


In [84]:
#We normalize the matrix
costs = (costs - np.mean(costs)) / np.std(costs)
print(costs)

[[ -0.17126667  -0.17126667  -0.17126667 ...,  -0.17126667  -0.17126667
   -0.17126667]
 [ -0.17126667  63.6612957   -0.17126667 ...,  -0.17126667  -0.17126667
   -0.17126667]
 [ -0.17126667  -0.17126667  29.86758621 ...,  -0.17126667  -0.17126667
   -0.17126667]
 ..., 
 [ -0.17126667  -0.17126667  -0.17126667 ...,  37.37729943  -0.17126667
   -0.17126667]
 [ -0.17126667  -0.17126667  -0.17126667 ...,  -0.17126667  41.13215604
   -0.17126667]
 [ -0.17126667  -0.17126667  -0.17126667 ...,  -0.17126667  -0.17126667
   59.90643909]]


In [47]:
costs = cost_function(7)
costs[7] = 0

In [48]:
Movies.iloc[np.argmax(costs)]

title                                                         Sea of Love
budget                                                           19000000
genres                                                Drama,Crime,Mystery
keywords                new york,alcohol,sex,detective,newspaper,nudit...
original_language                                                      en
original_title                                                Sea of Love
overview                Seen-it-all New York detective Frank Keller is...
popularity                                                        11.4732
production_companies                       Universal Pictures Corporation
production_countries                             United States of America
release_date                                                   1989-09-15
revenue                                                         110879513
runtime                                                               113
spoken_languages                      

In [86]:
np.savez_compressed("../../Datasets/costs.npz", costs, delimiter=",")

In [76]:
np.savez_compressed("../../Datasets/cosine_compressed.npz", cosine_distances, cosine = cosine_distances)

In [89]:
final_cost = cosine_distances * costs
np.savez_compressed("../../Datasets/finalcosts.npz", final_cost, cost = final_cost)

In [75]:
cosine_npz = np.load("../../Datasets/costs.npz")
print(cosine_npz['cosine'])

[[ 1.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          1.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          1.         ...,  0.02505487  0.          0.02338262]
 ..., 
 [ 0.          0.          0.02505487 ...,  1.          0.06506     0.03456506]
 [ 0.          0.          0.         ...,  0.06506     1.          0.03035884]
 [ 0.          0.          0.02338262 ...,  0.03456506  0.03035884  1.        ]]
